In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from googlesearch import search
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import requests
import signal
import re
from contextlib import contextmanager


class TimeoutException(Exception): 
    pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

topic = input("Enter topic: ")
texts = []
for j in search(topic, tld="com", num=10, stop=10, pause=2):
    try:
        with time_limit(30):
            content = requests.get(j)
            soup = BeautifulSoup(content.text, 'lxml')
            p_tags = soup.find_all('p')
            for paragraph in p_tags:
                texts.append(paragraph.text)
    except TimeoutException as e:
        continue

words = []

for text in texts:
    tokens = word_tokenize(text) 
    lemmatizer = WordNetLemmatizer() 
    tokens = [lemmatizer.lemmatize(token) for token in tokens] 
    tokens = [re.sub('[^\w\s]','',token) for token in tokens]
    tokens = [token.lower() for token in tokens]

    stop_words = set(stopwords.words('english'))
    filtered_words = [w for w in tokens if not w.lower() in stop_words]
    filtered_words = [w for w in filtered_words if w != "wa" and w != topic]
    words.extend(filtered_words)

eng_words = set(nltk.corpus.words.words())
words = [word for word in words if word in eng_words]
word_occurrences = [(word, words.count(word)) for word in set(words)]
word_occurrences.sort(key=lambda x:x[1], reverse=True)

# print the first 10 elements from the list
for pair in word_occurrences[:10]:
    print(pair[0]);
    


Enter topic: fan
setting
help
space
area
cool
model
pleasant
cooling
size
living
